<a href="https://colab.research.google.com/github/waheed444/AgenticAI-Playground/blob/main/03_Video_Audio_Analysis_mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-genai

### Install Prerequisites

In [2]:
!pip install gtts -q

In [3]:
!pip install playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=716c2e08cc1f01eb15ab024271036f48bb0822e7fe903e6fa4f98ea84cf9bc69
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


### Configure API Key

In [4]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
if(GEMINI_API_KEY):
  print("API Key is  found")
else:
  print("API Key is not found")

API Key is  found


### Setup gemini-2.0-flash Model

In [5]:
from google import genai
from google.genai import Client
client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)
model: str = "gemini-2.0-flash-exp"


### Import saved video from Drive

In [12]:
video_file_name = "/content/my_video.mp4"

### Upload the video to a AI-Server using a **client** library

In [10]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video('my_video.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/guyl32m73e6i


### Analyse the video and give their description

In [11]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(file_uri=video.uri or "", mime_type=video.mime_type or ""),
                # Add the text parameter here
                Part(text="This is the text you want the model to process.")
            ]
        )
    ]
)
print(response.text)

Okay, here's a breakdown of the provided video and the young man’s introduction:

**Overall:**

The man is recording himself to practice giving an interview introduction. He's wearing a suit, which gives him a professional appearance.

**Key Points in his Introduction:**

*   **Greeting:** He begins with "Assalam-o-Alaikum," which is a respectful greeting in Islam.
*   **Purpose of the video:** He asks viewers to tell him his mistakes in his interview, and how to do better.
*   **Introduction:** He states his name, his father’s name, and tells the profession of both his parents.
*   **Family:** He mentions he has three siblings and that he is the second born.

**Areas for Improvement (based on his request):**

*   **Less Focus on Family:** In a professional interview introduction, focus on yourself, your experience, your education, and your skills first. Mentioning family could be brief or not at all.
*   **Confidence:** While he is polite and makes eye contact, he could exude more con

### Analyse the text in **" "** and convert in into audio

In [13]:

from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

tts = gTTS(text = "\"Hey, I am Waheed Ahammad .I am Computer Science Student and currently I am learning Generative AI from PIAIC in UMT Lahore."
, lang='en')
tts.save("test.mp3")

with open("output.mp3", "wb") as f:
  for chunk in tts.stream():
    f.write(chunk)

display(Audio("output.mp3", autoplay=True))